## 라이브러리 호출 및 기본 설정

In [ ]:
# pandas 대신 pd 로 쓰자는 의미. mte 로 써도 mte 로 사용 가능.
import pandas as pd

# scipy 에 속하는 모듈 라이브러리. 주로 행렬, 수학적 데이터 등을 다룰 때 사용한다.
import numpy as np

# seaborn 은 그래프를 그리는 라이브러리 중 하나라 하더라... 사용법은 matplot 이랑 유사하다.
import seaborn as sns

# 시각화 라이브러리 호출
import matplotlib.pyplot as plt

# matplot 으로 시각화 표출 진행이 가능하다. 폰트를 설정해준다.
plt.rc('font', family = 'AppleGothic')

# 마이너스 값 (음수) 표현에 문제가 있는 듯하다. 깨지는 걸 방지하기 위한 설정.
plt.rc('axes', unicode_minus = False)

## 데이터 호출 및 기본적인 pandas 함수 작동

In [ ]:
df = pd.read_csv('./소상공인시장진흥공단_상가업소정보_의료기관_201909.csv', low_memory = False, encoding='cp949')

# 몇 행, 몇 열 인지 파악하기 위한 튜플 자료구조로 나온다.
df.shape

# shift + tab 으로 docstring 을 사용할 수 있다! (Jupyter Notebook 한정)

In [ ]:
# 위에서 3행 데이터를 가져온다.
df.head(3)

In [ ]:
# 아래에서 3행 데이터를 가져온다.
df.tail(3)

In [ ]:
# CSV 에서 취합된 데이터에 대한 메타 데이터 (혹은 필드) 정보를 열람할 수 있다.
df.info()

# 필드 값은 컬럼 이름, 값 존재 Cnt, 데이터 타입 이렇게 취합되어 나온다.

In [ ]:
# 데이터 타입 종류들만 열람이 가능하다.
df.dtypes

## 결측치 해결 및 제거하기

- null 값에 대한 제약사항을 해결하자.
- NaN vs null 차이 -> 숫자가 무효해도 결측치로 인정해주는지 확인해볼 것.
- 머신러닝 등 데이터 분석 사용 시, 결측치 값은 웬만하면 제거해주는 편이 좋다.

In [ ]:
# 데이터 별 null 값 카운트
df.isnull()

In [ ]:
# 아래 로직을 숫자로 반환 가능한 이유? -> Python 에서 True == 1 이 성립된다.
null_cnt = df.isnull().sum() # 컬럼 별 null 개수 반환

# 아래 결과는 Series 로 나올 것으로 추정된다.
print(type(null_cnt))

null_cnt

In [ ]:
# plot() 은 기본으로 line 으로 그려진다. bar 는 막대, barh 는 가로막대.
# rot 는 label 회전 각도. (rotate 에서 따온 모양.)
# figsize 는 그래프의 크기를 조정한다. (width, height 인치 기준.)
null_cnt.plot.barh(figsize=(5, 7))

In [ ]:
# 컬럼 별 null 값 목록 결과를 data frame 으로 반환한다.
# 현재까지 출력된 데이터 개수 (null_cnt) 는 Series 기반으로 취합되어 있었다.
df_null_cnt = null_cnt.reset_index()

In [ ]:
# 컬럼 이름을 각각 설정할 수 있나 보다. 그러나 컬럼 수가 다르면 무용지물 아닐까?
df_null_cnt.columns = ['컬럼명', '결측치수']

# df_null_cnt.columns = ['컬럼명', '결측치수', '11수']

# 기본적으로 5 개의 행이 나오는 것으로 추론된다.
df_null_cnt.head()

In [ ]:
# sort_values, sort_index 차이에 대해 분명히 할 것. sort_values 는 컬럼 값에 대한 정렬.
# 상위 10 개를 출력한다.
df_null_cnt \
    .sort_values(by = ['결측치수'], ascending = False) \
    .head(10) 

In [ ]:
# NaN 이 여기서는 Not a Number 로 생각하면 된다. 즉, 결측치.

# 컬럼으로 가져오려면 아래와 같이 작성한다. (인덱싱) 결과값은 Series 로 가져온다. 
df['지점명']

# 슬라이딩 및 여러 인덱싱은 자동으로 DataFrame 이 될 것이다.
df[['지점명', '상호명']]

In [ ]:
# 아래와 같이 질의 중에도 슬라이싱 및 인덱싱이 가능한 것으로 추정된다.
df_null_cnt_top = df_null_cnt \
    .sort_values(by = ['결측치수'], ascending = False) \
    .head(10) \
    ['컬럼명']

# 상위 10 개의 결측치 컬럼명의 목록을 출력한다.
df_null_top_columns = df_null_cnt_top.to_list()

df_null_top_columns

In [ ]:
# 결측지 맛집 Top 10 을 선정한 컬럼들로 데이터 목록을 조회한다.
df[df_null_top_columns].head()

# 결측치가 많은 데이터들에 대해 전부 제거한다.
# axis 는 0 일 때 행, 1 일 때 열.
print('삭제 전 -> ', df.shape)

# implace 를 사용해서 내부에 변경하는 경우도 있지만, 실제로는 immutable 하게 처리하는 것이 좋을 수 밖에 없다.
df = df.drop(df_null_top_columns, axis = 1)

print('삭제 후 -> ', df.shape)

In [ ]:
# 제거 뒤, info 를 통해 줄어든 데이터 용량을 확인한다.
df.info()

## 수치 (연속형), 문자열 (범주형) 데이터 요약하기 (기술통계)

- 데이터 타입이 int, float 등 numeric 한 데이터들에 대해서는 기술통계 값들을 구할 수 있다.
- 데이터 타입이 Object 인 데이터들에 대해서는 최빈도 값들을 구할 수 있다.

In [ ]:
print(df["위도"].mean()) # 평균
print(df["위도"].median()) # 중앙값
print(df["위도"].max()) # 최댓값
print(df["위도"].min()) # 최소값
print(df["위도"].count()) # 개수 (비어 있는 값을 제외한 개수!)

In [ ]:
# 1 사분위, 2 사분위 등등의 Quantite 값들도 존재한다.
# 단일 행은 Series 객체로 반환된다.
df["위도"].describe() # 위 내용을 전부 요약한다.

# 복수 행은 DataFrame 객체로 반환된다.
df[["위도", "경도"]].describe()

In [ ]:
# 문자열에 대한 Not Null 개수, Distinct 개수, 가장 많은 문자열 개수 등이 포함되어 있다.
# Number 는 숫자에 대한 계산, Object 는 문자열에 대한 계산.
df.describe(include = 'object')

In [ ]:
# 문자열 데이터에 대해 요약해보자.
# 중복 값들을 제거한 데이터 목록 (numpy 배열로 반환되는 모양이다.)
df["상권업종대분류명"].unique()

In [ ]:
# 중복 값들을 제거한 데이터 목록의 개수
df["상권업종대분류명"].nunique()

In [ ]:
# 중분류, 소분류도 같이 해보자.
df["상권업종중분류명"].unique()
df["상권업종중분류명"].nunique()

# nunique 도 이렇게 구현한 모양인 거 같다.
print(len(df["상권업종중분류명"].unique()))

In [ ]:
# 각 컬럼에 해당하는 값들을 Counting 하고, Desc 하게 뿌려준다.
city_normalize = df["시도명"].value_counts()

city_normalize.plot.barh()

In [ ]:
# 각 컬럼에 해당하는 값들을 Counting 한 비율을 구해준다.
# 이런 거는 솔직히... 원그래프 그리기 딱 좋은 거 같다.
city_normalize = df["시도명"].value_counts(normalize = True)

# 서울, 경기도 둘 중 어느 곳이 많은지 감이 잡히기 어렵다.
city_normalize.plot.pie(figsize = (7, 7))

In [ ]:
# seaborn 을 사용한 그래프 그리기
# seaborn 은 시각화 기능에 대한 통계적인 요소들을 간단히 적용할 수 있는 기능들을 보유하고 있다.
sns.countplot(data=df, y="시도명")

In [ ]:
cnt = df["상권업종중분류명"].value_counts()
per = df["상권업종중분류명"].value_counts(normalize=True)

In [ ]:
# 막대 그래프를 그릴 때, 각도가 90 도 기본으로 되어 있는 거 같다.
# 막대 그래프는 데이터 개수에 대한 판단이 필요할 때 주로 사용한다.
# barh 는 가로 그래프 (horizon)
cnt.plot.bar(rot = 0, grid = True)

In [ ]:
# 원 그래프는 데이터 비율에 대한 판단이 필요할 때 주로 사용한다.
per.plot.pie()

## 데이터 색인하기

- 특정 데이터들에 대한 모음을 구성할 때 사용한다.
- 여러 조건들에 대해서는 &, | 연산자 등으로 취합해서 사용하는 양상이다.

In [ ]:
# 상권업종중분류명 -> 약국/한약방
# filter 조건은 df 안에 아래와 같이 관계 대수를 사용하여 정의하면 되겠다.
# 데이터 추출 결과는 웬만하면... 복사해서 사용하는 것이 좋겠다... 아무래도 Call By Reference 의 한계도 있는 거 같다.
df_medical = df[df['상권업종중분류명'] == '약국/한약방'].copy()

df_medical.head()

In [ ]:
# 이렇게만 작성하는 것은 각 데이터 Row 별 해당 조건을 충족하는지 확인하기 위한 Series 객체를 반환한다.
m = df["상권업종대분류명"] == "의료"

# df.loc 를 사용해서 행, 열을 함께 가져올 수 있다.
# 아래와 같이 [][] 를 쓰지 말고, [ A, [B, ... ] ... ] 이렇게 사용하는 것이 좋겠다.
df.loc[m, "상권업종중분류명"].value_counts()

In [ ]:
# 유사의료업만 따로 모아보기
# shape 는 행, 열 순으로 개수를 뽑아본다.
df[df["상권업종중분류명"] == "유사의료업"].shape

In [ ]:
# 상호명 중 가장 흔히 사용하는 가게 이름 개수 뽑기 (예를 들어 서울에 보람약국이 있으면, 대전에도 보람약국이 있을 수 있다.)
# 아래와 같이 메소드 끼리 실행 순서를 정의하는 것이 메소드 체이닝. 마치 Functional Programming 이랑 같다.
df['상호명'].value_counts().head()

In [ ]:
df_medi = df[df['상권업종중분류명'] == '유사의료업'].copy()

print(df_medi.shape)

# 유사의료업에서 사용하는 가장 많은 상호명 상위 10 개 추출
df_medi['상호명'].value_counts().head(10)


In [ ]:
# 아래 두 가지 조건을 만족 시켜보자.
df['상권업종소분류명'] == '약국'
df['시도명'] == '서울특별시'

# 위들을 and 로 하면 되겠지? 는 아니다. 결과는 ambiguous 으로 애매한 것으로 나온다.
# 여기서는 어쩔 수 없이 & 을 사용해야 한다. 그리고 각 조건 당 소괄호로 체크해야 한다... Python 의 연산자로 사용하는 것이 아닌, 그들만의 연산자가 정의된 것으로 사료된다.
df_seoul_drug = df[(df['상권업종소분류명'] == '약국') & (df['시도명'] == '서울특별시')]

print(df_seoul_drug.shape)

df_seoul_drug.head(5)

In [ ]:
c = df_seoul_drug['시군구명'].value_counts()
c.head()

n = df_seoul_drug['시군구명'].value_counts(normalize=True)
n.head()

In [ ]:
c.plot.bar(rot = 60)

In [ ]:
df_seoul_hosp = df[(df['상권업종소분류명'] == '종합병원') & (df['시도명'] == '서울특별시')].copy()

# 해당 데이터에는 ~ 의원, ~ 매장 등등이 포함되어 있다...
df_seoul_hosp['시군구명'].value_counts()

In [ ]:
# '종합병원' 이란 문자열이 포함된 녀석들만 확인을 원하면 아래와 같이 작성한다.
df_seoul_hosp['상호명'].str.contains('종합병원')

# '종합병원' 문자열 포함을 안 시키면 ~ 으로 써야 한다. ! 가 아님을 유의하라.
~df_seoul_hosp['상호명'].str.contains('종합병원')

# 상위 조건들에 대해서는 변수들에 대해 같게 해야 한다. 조건 달을 때 유의하라.
# 텍스트 데이터 전처리 (즉, 종합병원 같은 이름을 추출하는 과정...) 를 시작하기 위해 우선 병원 이름 목록들을 출력한다.
df_seoul_hosp.loc[~df_seoul_hosp['상호명'].str.contains('종합병원'), '상호명'].unique()

# '꽃배달' 문자열이 포함된 경우를 빼기 위한 조건
~df_seoul_hosp['상호명'].str.contains('꽃배달')

# '의료기' 문자열이 포함된 경우를 빼기 위한 조건
~df_seoul_hosp['상호명'].str.contains('의료기')

In [ ]:
# 아래와 같은 종합병원의 ㅈ 자와 관련 없는 데이터들에 대해 검색하고, 인덱스를 가져온다.
drop_row = df_seoul_hosp[
    df_seoul_hosp['상호명'].str.contains('꽃배달|의료기|장례식장|상담소|어린이집')
].index

# python list 로 반환한다.
drop_row = drop_row.tolist()
drop_row

In [ ]:
# startswith, endswith 도 되나 보다...
drop_row2 = df_seoul_hosp[df_seoul_hosp['상호명'].str.endswith('의원')].index
drop_row2 = drop_row2.tolist()

drop_row2

In [ ]:
# 인덱스 목록들에 대해 합쳐준다. merge 를 하는 건 아니고... concat 같은 느낌이 든다.
drop_row = drop_row + drop_row2

len(drop_row)

In [ ]:
print(df_seoul_hosp.shape) # Before
df_seoul_hosp = df_seoul_hosp.drop(drop_row, axis=0)
print(df_seoul_hosp.shape) # After

In [ ]:
plt.figure(figsize=(15,4))

# 개수로 자동으로 정렬이 안 되니... order 값으로 설정해둘 것. (seaborn 한정)
sns.countplot(data = df_seoul_hosp, x = '시군구명', order = df_seoul_hosp['시군구명'].value_counts().index)

# 데이터 전처리에 대해서는... (예를 들어 ABC병원내편의점 등 문자열이 포함되어 있는 경우...) 정리하는 작업이 조금 지루해질 수 있는 것도 감안해야 할 것이다.

## 위경도 데이터 Scatter Plot (산점도) 실습

In [ ]:
df_seoul = df[df['시도명'] == '서울특별시'].copy()

print(df_seoul.shape)

df_seoul['시군구명'].value_counts()

# countplot 는 seaborn 에 있다. x 축, y 축을 따로 정의하는 것임을 유의하라.
plt.figure(figsize = (15, 4))
sns.countplot(data = df_seoul, x="시군구명")

In [ ]:

# 산점도 모습이 마치 서울특별시 지도랑 비슷한 양상으로 보인다.
df_seoul[['경도', '위도', '시군구명']].plot.scatter(x = '경도', y = '위도', figsize = (8, 7), grid = True)

In [ ]:
# hue 는 색상을 다르게 하는 것이다.
plt.figure(figsize = (9, 8))

# 그래프가 각 구 마다 색상이 다른 양상으로 나온다.
sns.scatterplot(data = df_seoul, x = '경도', y = '위도', hue = '시군구명')

In [ ]:
# 그래프가 각각 다르게 나올 것이다.
sns.scatterplot(data = df_seoul, x = '경도', y = '위도', hue = '상권업종중분류명')

In [ ]:
# 너무 많이 불러오면 느릴 것으로 판단되니... 1000 개만 그려보자.
plt.figure(figsize = (16, 12))
sns.scatterplot(data = df, x = '경도', y = '위도', hue = '시도명')

## Folium 으로 위경도 지도 그리기

In [ ]:
import folium

lat = df_seoul_hosp['위도'].mean()
lot = df_seoul_hosp['경도'].mean()

print(lot, lat)

map = folium.Map(
    location = [lat, lot], 
    zoom_start = 12 
)

for n in df_seoul_hosp.index:
    name = df_seoul_hosp.loc[n, '상호명']
    address = df_seoul_hosp.loc[n, '도로명주소']
    popup = f"{name} - {address}"
    tlat = df_seoul_hosp.loc[n, '위도']
    tlot = df_seoul_hosp.loc[n, '경도']
    folium.Marker(
        location = [tlat, tlot], popup = popup
    ).add_to(map)

map


In [ ]:
df.count().plot.barh()